# Calculate Expected Annual Damage for all road segments in a NUTS-3 region.
This script calls the EAD_multi script, which carries out the computation of the Expected Annual Damage using (1) the output of the main model and (2) the flood protection data. Before running this notebook, make sure to have run the main model as well as the postprocessing AoI_FP step.

In [3]:
import sys
sys.path.append("..") #import folder which is one level higher

import os as os
import pandas as pd
from pathos.multiprocessing import Pool
import time as time
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

from postproc_functions import EAD_multi, NUTS_3_EU28_cleaned, NUTS_up
from utils_functions import load_config

The script runned for 1.6271781921386719 seconds.


### Run for one NUTS-3 region

In [ ]:
EAD_multi('DE224')

### Select regions to run the model for

In [ ]:
#RUN FOR THE EU-28
#regions = NUTS_3_EU28_cleaned()

#RUN FOR ONE COUNTRY
#regions = NUTS_up('CH',True)

#RUN FOR MULTIPLE COUNTRIES
#super_regions = ['LI','NO']
#regions = []
#for super_region in super_regions:
#    regions.extend(NUTS_up(super_region,True))

### Run for multiple regions using parallel processing

In [ ]:
start = time.time()

if __name__ == '__main__': 
    with Pool(5) as pool: 
        pool.map(EAD_multi,regions,chunksize=1)
    
print("The script runned for {} seconds.".format(time.time()-start))